

# Título   Chequear ternaria

## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Chequear en un número de filas si fueron bien clasificadas en clase_ternaria

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [24]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

casos_a_chequear=1000     # cantidad de filas a chequear

#verbose=0       # Sólo muestra cantidad de errores
verbose=1       # Sölo muestra los errores
#verbose=2       # Muestra todos los casos analizados

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [2]:
dataset_path = '~/buckets/b1/datasets'
dataset_file = 'competencia_02.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [3]:
#output_file='competencia_02.csv.gz'


In [4]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [5]:
import os
import pandas as pd

In [6]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [7]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'Leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.shape

Leyendo ~/buckets/b1/datasets/competencia_02.csv.gz


(4735593, 155)

In [9]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [21]:
columnas_de_interes = ['numero_de_cliente', 'foto_mes', 'clase_ternaria']

# Obtener una muestra aleatoria (puedes especificar el tamaño de la muestra con n)
df=df[columnas_de_interes]
muestra = df.sample(n=casos_a_chequear)
muestra['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    787
BAJA+1      111
BAJA+2      102
Name: count, dtype: int64

In [22]:
def chequear_fila (fila, df):
  clase_ternaria=fila['clase_ternaria']
  numero_de_cliente=fila['numero_de_cliente']
  mes=fila['foto_mes']
  mes_mas_1=sig_mes(mes, 1)
  mes_mas_2=sig_mes(mes, 2)
  existe1=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_1)].shape[0]>0
  existe2=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_2)].shape[0]>0
  if clase_ternaria=='CONTINUA':
    ok=existe1 and existe2
  elif clase_ternaria=='BAJA+1':
    ok=not existe1
  elif clase_ternaria=='BAJA+2':
    ok=existe1 and not existe2
  else:
    ok=False
  mensaje=f'Clte: {numero_de_cliente} {clase_ternaria} ok: {ok}   ex1: {existe1} ex2: {existe2}'
  return (ok, mensaje)

In [25]:
num=0
errores=0
for index, fila in muestra.iterrows():
  num=num+1
  ok, mensaje=chequear_fila(fila, df)
  if verbose==2 or (verbose==1 and not ok):
      print (num, mensaje)
  if not ok:
    errores=errores+1


11 Clte: 875855876 BAJA+1 ok: False   ex1: True ex2: True
13 Clte: 621958040 BAJA+2 ok: False   ex1: True ex2: True
27 Clte: 1208133168 BAJA+2 ok: False   ex1: True ex2: True
39 Clte: 594804307 BAJA+1 ok: False   ex1: True ex2: True
49 Clte: 516187616 BAJA+1 ok: False   ex1: True ex2: True
57 Clte: 512086494 BAJA+1 ok: False   ex1: True ex2: True
63 Clte: 1027307302 BAJA+2 ok: False   ex1: True ex2: True
68 Clte: 571894045 BAJA+1 ok: False   ex1: True ex2: True
72 Clte: 1197261344 BAJA+1 ok: False   ex1: True ex2: True
77 Clte: 895338195 BAJA+1 ok: False   ex1: True ex2: True
87 Clte: 432297426 BAJA+2 ok: False   ex1: True ex2: True
95 Clte: 560646438 BAJA+1 ok: False   ex1: True ex2: True
97 Clte: 620625005 BAJA+1 ok: False   ex1: True ex2: True
131 Clte: 788608930 BAJA+2 ok: False   ex1: True ex2: True
132 Clte: 271687382 BAJA+1 ok: False   ex1: True ex2: True
156 Clte: 794242174 BAJA+2 ok: False   ex1: True ex2: True
165 Clte: 1163639855 BAJA+2 ok: False   ex1: True ex2: True
176 Cl

In [ ]:
errores

In [ ]:
# Eliminar los campos creados
print (f'Eliminando campos creados')
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])

Eliminando campos creados


In [ ]:
ruta_nuevo_archivo = os.path.join(dataset_path, output_file)
# Guarda el DataFrame en el nuevo archivo
print (f'Guardando el archivo de salida')
df.to_csv(ruta_nuevo_archivo, index=False, compression='gzip')
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")

Guardando el archivo de salida
El archivo ha sido guardado en: ~/buckets/b1/datasets/competencia_02.csv.gz


In [ ]:
df['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    3768977
BAJA+1       486920
BAJA+2       479696
Name: count, dtype: int64

In [ ]:
df.shape[0]

4735593